In [ ]:
#|default_exp utils

# Utils

In [ ]:
#|export
import os
import re
import numpy as np
from fastcore.all import *

In [ ]:
#|export
def convert_uuids_to_indices():
    cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES", "")
    uuids = re.findall(r"\b[0-9a-fA-F]{8}(?:-[0-9a-fA-F]{4}){3}-[0-9a-fA-F]{12}\b", cuda_visible_devices)

    if uuids:
        indices = [str(i) for i in range(len(uuids))]
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(indices)

In [ ]:
#|export
def percent_rel_error_(N, N_mc, N_mc_t0):
    """
    Return the percentage relative error with respect to the MC simulation
    See Eq. 23 of the paper by Giudici et al. (2023), 'Space debris density 
    propagation through a vinite volume method'
    Input:
        N: Predicted number of in-orbit elements over time
        N_mc: Number of in-orbit elements from the MC simulation over time
        N_mc_t0: Number of in-orbit elements from the MC simulation at t0
    Output:
        The percentage relative error (float)
    """
    return 100 * np.abs(N_mc - N) / N_mc_t0

In [ ]:
# Test
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([1, 2, 3, 4, 5])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.zeros_like(res))

# Test 2
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([2, 4, 6, 8, 10])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.array([ 50., 100., 150., 200., 250.]))



In [ ]:
#|eval: false
#|hide
from nbdev import *
nbdev_export()